In this notebook, we are going to perform top down market analysis to determine which stocks should be bought. Before we consider buying individual stocks, we have to analyze the overall market condition using dual momentum theory. Based on the dual momentum theory, we only invest in assets when their prices are going up or in uptrend.

In [1]:
import json
import momentum
import pyticker
import dividend
import pandas as pd
import yfinance as yf
import datetime as dt

## Global Macro Momentum

We can compute previous 12 months' momentums of some of top ETFs in different asset classes to observe the overall market movements. I've written helper functions to compute equally weighted momentum in momentum.py which is imported above. Below, we used the avearge of previous 1 month return, 3 months return, 6 months return and 12 months return to measure the momentum.

In [ ]:
global_macro = ['SPY', 'QQQ', 'TLT', 'IEF', 'GLD', 'DBC']
start_date = dt.datetime(1970,1,1)
end_date = dt.datetime.today()
momentum_df = momentum.calculate_equal_weight_momentum(global_macro, start_date, end_date, [1,3,6,12])
momentum_df

Based on the calcution above, we can see that QQQ has been the strongest market over the last year with 21.7% positive returns. Next, DBC and SPY have also around 21% returns. In contrast, Gold and US Bonds have negative returns over the last year. Clearly, equity markets have been much stronger than Gold and US Bonds markets.

The underlying assumption is that if we start investing today, we should consider buying assets in equities or DBC rather than Gold or US Bonds as equities and DBC have higher momentum scores.

## US Sector Momentum

### 1. Extract Sector Table from etfdb.com

In [2]:
url = 'https://etfdb.com/etfs/sector/'
df = pd.read_html(url)[0].drop([11], axis=0)

sectors = list(df['Sector'])
sectors = [x.lower() for x in sectors]
sectors = [x.replace(' ', '-') for x in sectors]
sectors = [x.replace('discretionary', 'discretionaries') for x in sectors]
#sectors

### 2. Retrieve Top ETFs from Each Sector

Assumption here is that ETFs with the largest AUMs in each sector contains major companies in each sector, and their performance are reflected on the performance of ETFs. Of course, performance of individual stocks may vary from the overall performance of ETFs, but it's good to measure the overall sentiment of sector by analyzing the price performance of top ETFs.

In [ ]:
top_etfs = {}
for s in sectors:
    sector_url = url + s + '/'
    sector_data = pd.read_html(sector_url)[0]
    etf_symbol = sector_data.loc[0, 'Symbol']
    etf_name = sector_data.loc[0, 'ETF Name']
    etf_industry = sector_data.loc[0, 'Industry']
    etf_aum = sector_data.loc[0, 'Total Assets ($MM)']
    top_etfs[s] = {}
    top_etfs[s]['symbol'] = etf_symbol
    top_etfs[s]['name'] = etf_name
    top_etfs[s]['industry'] = etf_industry
    top_etfs[s]['aum'] = etf_aum

In [ ]:
sector_data = {
    'Symbol': [],
    'Sector': [],
    'Industry': [],
    'AUM': []
}

for etf in top_etfs.keys():
    symbol = top_etfs[etf]['symbol']
    sector = etf
    industry = top_etfs[etf]['industry']
    aum = top_etfs[etf]['aum']

    sector_data['Symbol'].append(symbol)
    sector_data['Sector'].append(sector)
    sector_data['Industry'].append(industry)
    sector_data['AUM'].append(aum)

### 3. Sector Momentum Calculation

sector_momentum = momentum.calculate_equal_weight_momentum(sector_data['Symbol'], start_date, end_date, [1,3,6,12])

sector_df = pd.DataFrame(sector_data)
sector_df.set_index('Symbol', inplace=True)
sector_df['EW_MOMENTUM'] = sector_momentum['EW_MOMENTUM']
sector_df.sort_values(by='EW_MOMENTUM', ascending=False)

Based on the momentum calculation above, Oil & Gas sector has the highest momentum score with 44.8% positive returns and Gold industry has the lowest momentum score with 4.96% positive returns.

## US Individual Stocks

S&P 500 -> 25 Years Consecutive Dividend Payout -> Dividend Growth greater than 8% -> uptrend

### 1. Extract & Save Symbol Information

In [3]:
# sp500 = pyticker.get_symbols_by_index('S&P 500')
# div_sp500 = dividend.calculate_historical_annual_dividends(sp500, 25)
# dow = pyticker.get_symbols_by_index('DOW JONES')
# div_dow = dividend.calculate_historical_annual_dividends(dow, 25)

In [4]:
# with open('./historical_div_sp500.json', 'w') as fp:
# #     json.dump(div_sp500, fp)
# with open('./historical_div_dow.json', 'w') as fp:
#     json.dump(div_dow, fp)

In [5]:
with open('./historical_div_dow.json', 'r') as fp:
    div_dow = json.load(fp)

In [11]:
data_dict = {
    'Symbol': [],
    'Dividend_Growth': [],
    'Dividend_Yield': [],
    'Consecutive_Yrs': []
}

yrs = 15

with open('./historical_div_dow.json', 'r') as fp:
    div_dow = json.load(fp)

for symbol in list(div_dow.keys()):
    print(symbol)
    data_dict['Symbol'].append(symbol)

    data_dict['Dividend_Growth'].append(dividend.calcualte_avg_dividend_growth(div_dow, symbol, yrs))

    data_dict['Dividend_Yield'].append(dividend.calculate_current_dividend_yield(div_dow, symbol))

    data_dict['Consecutive_Yrs'].append(div_dow[symbol][1])

MMM
AXP
BA
KO
HD
IBM
INTC
JNJ
JPM
MRK
PG
TRV
WBA
HON


In [22]:
df = pd.DataFrame(data_dict)
df.set_index('Symbol', inplace=True)

In [23]:
start_date = dt.datetime(1970,1,1)
end_date = dt.datetime.today()
mom = momentum.calculate_equal_weight_momentum(list(df.index), start_date, end_date, [1,3,6,12])

In [24]:
mom

,Name,1M_Return,3M_Return,6M_Return,12M_Return,EW_MOMENTUM
Symbol,,,,,,
BA,Boeing Company (The),0.201453,0.189947,0.541329,0.707925,0.410163
JPM,JP Morgan Chase & Co.,0.034382,0.206629,0.607449,0.753646,0.400526
WBA,"Walgreens Boots Alliance, Inc.",0.145420,0.389753,0.561303,0.253983,0.337615
AXP,American Express Company,0.048833,0.177426,0.426007,0.686760,0.334756
HD,"Home Depot, Inc. (The)",0.188987,0.156399,0.112087,0.672755,0.282557
HON,Honeywell International Inc.,0.072745,0.025063,0.330740,0.658688,0.271809
TRV,"The Travelers Companies, Inc.",0.033677,0.071454,0.398938,0.545144,0.262303
MMM,3M Company,0.100651,0.111435,0.223301,0.462846,0.224558
INTC,Intel Corporation,0.052978,0.292422,0.252546,0.213343,0.202822


In [25]:
df['EW_MOMENTUM'] = mom['EW_MOMENTUM']

In [26]:
df

,Dividend_Growth,Dividend_Yield,Consecutive_Yrs,EW_MOMENTUM
Symbol,,,,
MMM,0.094810,0.029271,51,0.224558
AXP,0.126343,0.011686,44,0.334756
BA,0.119774,0.008711,59,0.410163
KO,0.077419,0.030031,59,0.092608
HD,0.210177,0.018408,32,0.282557
IBM,0.154163,0.045350,58,0.146801
INTC,0.157780,0.020722,29,0.202822
JNJ,0.084425,0.023891,57,0.123428
JPM,0.220355,0.023914,37,0.400526


In [27]:
df = df.sort_values(by='EW_MOMENTUM', ascending=False)
df

,Dividend_Growth,Dividend_Yield,Consecutive_Yrs,EW_MOMENTUM
Symbol,,,,
BA,0.119774,0.008711,59,0.410163
JPM,0.220355,0.023914,37,0.400526
WBA,0.155128,0.034265,36,0.337615
AXP,0.126343,0.011686,44,0.334756
HD,0.210177,0.018408,32,0.282557
HON,0.107212,0.015748,51,0.271809
TRV,0.072474,0.021413,34,0.262303
MMM,0.094810,0.029271,51,0.224558
INTC,0.157780,0.020722,29,0.202822
